In [53]:
#Dataset goes from 1st January, 2016 to 2nd October, 2017

import pandas as pd

df_elliptic = pd.read_csv("Data\elliptic_bitcoin_dataset\elliptic_txs_classes.csv")
df_unkw = df_elliptic[df_elliptic["class"] == "unknown"]
df_illicit = df_elliptic[df_elliptic["class"] == "1"]


df_deanomyzed = pd.read_csv("Data\99deanomyzed.csv")
tx_unkw_set = set(df_deanomyzed["txId"]).intersection(set(df_unkw["txId"]))
tx_illicit_set = set(df_deanomyzed["txId"]).intersection(set(df_illicit["txId"]))

finaldf = df_deanomyzed[df_deanomyzed["txId"].isin(tx_unkw_set)]
finaldf2 = df_deanomyzed[df_deanomyzed["txId"].isin(tx_illicit_set)]


print(finaldf)
print(finaldf2)

             txId                                        transaction
0       230325127  d6176384de4c0b98702eccb97f3ad6670bc8410d9da715...
1       230325139  300c7e7bb34263eae7ff8b0a726d5869bf73d71081490c...
2        86875675  7c790a31090462d720a172b3f55a51af2514971070db66...
3       230325147  052ccc5d925e3b1bf59e9375a471d97aa8df2208336428...
4       230325154  82666f65b865a0ada143fb460f38916618bf45e3132f07...
...           ...                                                ...
202799  157659046  18245b5afbd4f703e7c889978f133da3864362bc9f49ef...
202800  157659306  799589bc36bbb9a6b6523fe0cb1ed1e78d48c3f587f051...
202801  157668825  4d0a53797b5cc9a96294615d01be33448fe40ae42bfe59...
202802  125788182  9e3b1aba58c182ac80bef2e19690f71ce5c507560bfc55...
202803  157670868  7572580f10f8dfca7bac8dcfd82e81c23944e8149ab7c9...

[156759 rows x 2 columns]
             txId                                        transaction
693     184703182  10c11d060302ef4588a5ac952962dcb88607990b6e22ca...
695    

In [40]:
#Retrieval of the addresses' transactions linked to Ponzi schemes and ransomware
import requests
import time

with open("suspicious_addresses.txt") as f:
    for bitcoin_address in f: #loop each line that contains an address
        address_out_of_range = 0 #if address' txs are not between 2016-01-01 03:50 and 2017-10-02 19:21 we set the variable to 1
        bitcoin_address = bitcoin_address.rstrip()
        URL_address = "https://blockstream.info/api/address/" + bitcoin_address
        print(bitcoin_address)
        try: 
            r = requests.get(URL_address, timeout=5)
            print(r.status_code)
        except requests.exceptions.Timeout as e:
            with open("timeout_addresses.txt", "a") as f1:
                print("timeout: " + bitcoin_address + "\n")
                line = bitcoin_address + '\n'
                f1.write(line) #in case we get a timeout, we save the address to try again later

        if(r.status_code == 200):
            total_tx = r.json()['chain_stats']['tx_count'] #total number of tx in which the address has been involved
            tx_left = total_tx
            
            #Get transaction history for the specified address, sorted with newest first.
            #Returns the first 25 confirmed transactions.
            URL_last_tx = "https://blockstream.info/api/address/" + bitcoin_address + "/txs" 
            counter = int(total_tx / 25) + 1
            for i in range (counter):
                if (i == 0):
                    r = requests.get(URL_last_tx, timeout=5)
                
                else:
                    #Returns 25 transactions per page. More can be requested by specifying the last txid seen by the previous query.
                    URL_more_tx = "https://blockstream.info/api/address/" + bitcoin_address + "/txs/chain/" + tx_hash 
                    r = requests.get(URL_more_tx, timeout=5)

                if(tx_left >= 25):        
                    for j in range (25):
                        tx_hash = r.json()[j]['txid']
                        tx_time = r.json()[j]['status']['block_time']
                        tx_left-=1
                        if(tx_time > 1451616540 and tx_time < 1506964860): #tx between 2016-01-01 03:50 and 2017-10-02 19:21
                            with open('tx_from_suspicious_addresses.csv','a') as fd:
                                row = str(tx_time) + "," + str(tx_hash) + "\n"
                                fd.write(row)
                        else:
                            address_out_of_range = 1
                            break
                else:
                    #In case there are less than 25 transactions left, we avoid going out of range
                    for j in range (tx_left):
                        tx_hash = r.json()[j]['txid']
                        tx_time = r.json()[j]['status']['block_time']
                        if(tx_time > 1451616540 and tx_time < 1506964860): #tx between 2016-01-01 03:50 and 2017-10-02 19:21
                            with open('tx_from_suspicious_addresses.csv','a') as fd:
                                row = str(tx_time) + "," + str(tx_hash) + "\n"
                                fd.write(row)
                        else:
                            address_out_of_range = 1
                            break
                
                if(address_out_of_range == 1):
                    break
                else:
                    time.sleep(5) #To avoid a DoS attack to the API

178BzARKjkszrTyx4TxBKHhzGLZijdE26e
200
13aJVen6GVWrGmyUx26bqBoxSefJqbiYcq
200
1GQS1d42WRAJUBZvBC4gdx8MvDktMxeDBq
200


In [2]:
import pandas as pd

df_new_txs = pd.read_csv("tx_from_suspicious_addresses.csv")
df_deanomyzed = pd.read_csv("Data\99deanomyzed.csv")

repeated_tx_set = set(df_deanomyzed["transaction"]).intersection(set(df_new_txs["transaction"]))
repeated_df = df_deanomyzed[df_deanomyzed["txId"].isin(repeated_tx_set)]
print(repeated_df)

Empty DataFrame
Columns: [txId, transaction]
Index: []
